In [2]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 45)
pd.set_option('display.max_rows', 50)

item_master = pd.read_excel('../data/inzi/Item Master_08_18_2021 0317.xls', header=1)
material_history = pd.read_excel('../data/inzi/Material Movement History_08_19_2021 1026.xls', header=1)

material_history['Material'].value_counts()

temp_item_master = item_master[['Material', 'Material Type', 'Procurement']]

temp_item_master.loc[temp_item_master['Material'] == 'BN61-08214A']

step_one = pd.merge(material_history, temp_item_master, on="Material", how="left")


# step_one['Input'] = step_one['Quantity'] if step_one['Quantity'] > 0 else None
# step_one['Output'] = -step_one['Quantity'] if step_one['Quantity'] < 0 else None

def create_quantity(columns):
    if columns > 0:
        return [columns, 0]
    else:
        return [0, -columns]

step_one['Input'] = step_one['Quantity']
step_one['Output'] = step_one['Quantity']

# step_one['Input'] = [value for value in step_one['Input'] if step_one['Input'].all() > 0]

step_one.rename(columns={
    'Reference': 'Order Category',
    'Unnamed: 13': 'Order Data',
    'Unnamed: 14': 'Master Category',
    'Unnamed: 15': 'Master Data',
    'Unnamed: 16': 'Remark'
}, inplace=True)

step_one = step_one.iloc[1: , :]


# step_one.loc[step_one['Input'] > 0]
def temp_filter(values):
    return values if values > 0 else 0

# def filter_inout(row):
#     return row['Quantity'] if row['Quantity'] > 0 else 0

# step_one['input'] = step_one.apply(filter_inout, axis=1)

# pd.set_option('mode.chained_assignment', None)

step_one.loc[step_one['Quantity'] > 0, 'Input'] = step_one['Quantity']
step_one.loc[step_one['Quantity'] > 0, 'Output'] = 0
step_one.loc[step_one['Quantity'] <= 0, 'Input'] = 0
step_one.loc[step_one['Quantity'] <= 0, 'Output'] = -step_one['Quantity']

cols = step_one.columns.tolist()

cols = cols[:10] + [cols[-2], cols[-1]] + cols[10:-2]

step_one = step_one[cols]

step_one['Account code'] = [int(x) for x in (step_one['Movement Type'].str[:3])]

cols = step_one.columns.tolist()

cols = cols[:7] + [cols[-1]] + cols[7: -1]

step_one = step_one[cols]

summary = step_one.pivot(index=["Unnamed: 0" ,"Material"], columns="Account code", values="Quantity")
summary.reset_index(inplace=True)
del summary['Unnamed: 0']
aggregation_functions = {}
columns_order = [101, 102, 103, 321, 323, 327, 343, 344, 401, 602, 701,
                     720, 801, 809, 201, 261, 555, 601, 609, 702, 712, 721, 803]
for column in summary.columns:
    if isinstance(column, int):
        aggregation_functions[column] = 'sum'
        if column in [321, 323, 327, 343, 344, 401]:
            aggregation_functions[column] = 'first'

final_col = []
for col in columns_order:
    if col in summary.columns:
        final_col.append(col)

summary = summary.groupby(summary['Material']).aggregate(
        aggregation_functions)

for column in columns_order:
    if column not in summary.columns:
        summary[column] = np.nan
summary.fillna(0, inplace=True)
summary = summary[columns_order]
summary['IN'] = summary[[101, 602, 720, 801, 809]].sum(axis=1)
summary['OUT'] = summary[[102, 201, 261,
                            555, 601, 609, 721, 803]].sum(axis=1)
summary.reset_index(inplace=True)
summary.index.name = None

In [3]:
summary

Account code,Material,101,102,103,321,323,327,343,344,401,602,701,720,801,809,201,261,555,601,609,702,712,721,803,IN,OUT
0,0103-010237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,300000.0,0.0,0.0,0.0,0.0,0.0,0.0,-483120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-483120.0
1,0103-011284,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,250000.0,0.0,0.0,0.0,0.0,0.0,0.0,-1208405.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1208405.5
2,0103-011475,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,600000.0,0.0,0.0,0.0,0.0,0.0,0.0,-585578.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-585578.4
3,0103-011580,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,225000.0,0.0,0.0,0.0,0.0,0.0,0.0,-852816.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-852816.0
4,0103-011600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100000.0,0.0,0.0,0.0,0.0,0.0,0.0,-439560.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-439560.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,IZ64-04515B,1467.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1467.0,0.0
305,IZ64-04516A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-457.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-457.0
306,IZ64-04519A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-241.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-241.0
307,MD63-00103A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2000.0,0.0,0.0,0.0,0.0,0.0,0.0,-5966.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5966.0


In [ ]:
columns_name = {
    'H': 'Account code',
    'p': 'Order Category',
    'W': 'Material Type',
    'X': 'Procurement'
}

nhap_vao = [None] * 7
nhap_vao[0] = ((step_one['Account code'] == 101) & (step_one['Order Category'] == 'Purchase Order'))
nhap_vao[1] = (step_one['Account code'] == 602)
nhap_vao[2] = (step_one['Account code'] == 623)
nhap_vao[3] = (step_one['Account code'] == 701)
nhap_vao[4] = (step_one['Account code'] == 720)
nhap_vao[5] = (step_one['Account code'] == 801)
nhap_vao[6]= (step_one['Account code'] == 809)

xuat_ra = [None] * 9
xuat_ra[0] = ((step_one['Account code'] == 102) & (step_one['Order Category'] == 'Purchase Order'))
xuat_ra[1] = (step_one['Account code'] == 201)
xuat_ra[2] = (step_one['Account code'] == 261)
xuat_ra[3] = (step_one['Account code'] == 601)
xuat_ra[4] = (step_one['Account code'] == 609)
xuat_ra[5] = (step_one['Account code'] == 702)
xuat_ra[6] = (step_one['Account code'] == 712)
xuat_ra[7] = (step_one['Account code'] == 721)
xuat_ra[8] = (step_one['Account code'] == 803)

thuyen_chuyen = [None] * 2
thuyen_chuyen[0] = (step_one['Account code'].between(300, 400))
thuyen_chuyen[1] = (step_one['Account code'] == 401)

In [ ]:
# for i in range(len(nhap_vao)):
#     nhap_vao[i] = step_one.loc[nhap_vao[i]]
#     nhap_vao[i]['Tm_Rank'] = nhap_vao[i]['Material Type'].map(material_order_index)
#     nhap_vao[i].sort_values(['Tm_Rank', 'Procurement'], inplace = True)
#     nhap_vao[i].drop('Tm_Rank', 1, inplace = True)

In [ ]:
def condition_to_data(arr):
    material_type_order = ['ROH', 'HAWA', 'HALB', 'FERT']
    material_order_index = dict(zip(material_type_order, range(len(material_type_order))))
    for i in range(len(arr)):
        arr[i] = step_one.loc[arr[i]]
        arr[i]['Tm_Rank'] = arr[i]['Material Type'].map(material_order_index)
        arr[i].sort_values(['Tm_Rank', 'Procurement'], inplace = True)
        arr[i].drop('Tm_Rank', 1, inplace = True)

In [ ]:
condition_to_data(nhap_vao)
condition_to_data(xuat_ra)
condition_to_data(thuyen_chuyen)

In [ ]:
thuyen_chuyen[0]

In [ ]:
step_two = pd.concat(nhap_vao + xuat_ra + thuyen_chuyen)


In [ ]:
step_two

In [ ]:
condition_to_delete = ~(step_two['Account code'].isin([101, 102])) & (
    ((step_two['Material Type'] == 'ROH') & (step_two['Procurement'] == 'E')) | 
    ((step_two['Material Type'] == 'HAWA') & (step_two['Procurement'] == 'E')) |
    ((step_two['Material Type'] == 'HALB') & (step_two['Procurement'].isin(['E', 'X']))) | 
    ((step_two['Material Type'] == 'FERT') & (step_two['Procurement'].isin(['E', 'X'])))
)
step_two.drop(step_two[condition_to_delete].index, inplace = True)

In [ ]:
SCM_RAW = step_two

In [ ]:
nhap_vao = [None] * 2
nhap_vao[0] = (
    (step_one['Account code'] == 101) & 
    (step_one['Order Category'] == 'Purchase Order') & 
    (step_one['Material Type'] == 'HALB') & 
    (step_one['Procurement'].isin(['E', 'X']))
)
            
nhap_vao[1] = (
        (step_one['Account code'].isin([602, 623, 701, 720, 801, 809])) &
        (step_one['Material Type'] == 'HALB') & 
        (step_one['Procurement'].isin(['E', 'X']))
)

xuat_ra = [None] * 3
xuat_ra[0] = (
    (step_one['Account code'] == 102) & 
    (step_one['Order Category'] == 'Purchase Order') & 
    (step_one['Material Type'] == 'HALB') & 
    (step_one['Procurement'].isin(['E', 'X']))
)

xuat_ra[1] = (
        (step_one['Account code'].isin([201, 601, 609, 702, 712, 721, 803])) &
        (step_one['Material Type'] == 'HALB') & 
        (step_one['Procurement'].isin(['E', 'X']))
)

xuat_ra[2] = (
        (step_one['Account code'] == 261) &
        (
            ((step_one['Material Type'] == 'HALB') & 
            (step_one['Procurement'].isin(['E', 'X']))) |
            ((step_one['Material Type'] == 'FERT') & 
            (step_one['Procurement'].isin(['E', 'X'])))
        )
)

thuyen_chuyen = [None]
thuyen_chuyen[0] = (
        (step_one['Account code'].between(300, 402)) &
        (step_one['Material Type'] == 'HALB') & 
        (step_one['Procurement'].isin(['E', 'X']))
)

In [ ]:
step_one

In [ ]:
condition_to_data(nhap_vao)
condition_to_data(xuat_ra)
condition_to_data(thuyen_chuyen)

In [ ]:
SCM_WIP = pd.concat(nhap_vao + xuat_ra + thuyen_chuyen)

In [ ]:
SCM_WIP.loc[(
    (SCM_WIP['Account code'] == 261) &
    (SCM_WIP['Material Type'] == 'FERT') &
    (SCM_WIP['Location'].str.startswith('RW'))
), 'Location'] = None

In [ ]:
nhap_vao = [None] * 2
nhap_vao[0] = (
    (step_one['Account code'] == 101) & 
    (step_one['Order Category'] == 'Production Order') & 
    (step_one['Material Type'] == 'FERT') & 
    (step_one['Procurement'].isin(['E', 'X']))
)
            
nhap_vao[1] = (
        (step_one['Account code'].isin([602, 623, 701, 720, 801, 809])) &
        (step_one['Material Type'] == 'FERT') & 
        (step_one['Procurement'].isin(['E', 'X']))
)

xuat_ra = [None] * 2
xuat_ra[0] = (
    (step_one['Account code'] == 102) & 
    (step_one['Order Category'] == 'Production Order') & 
    (step_one['Material Type'] == 'FERT') & 
    (step_one['Procurement'].isin(['E', 'X']))
)

xuat_ra[1] = (
        (step_one['Account code'].isin([201, 601, 609, 702, 712, 721, 803])) &
        (step_one['Material Type'] == 'FERT') & 
        (step_one['Procurement'].isin(['E', 'X']))
)

thuyen_chuyen = [None]
thuyen_chuyen[0] = (
        (step_one['Account code'].between(300, 402)) &
        (step_one['Material Type'] == 'FERT') & 
        (step_one['Procurement'].isin(['E', 'X']))
)

In [ ]:
condition_to_data(nhap_vao)
condition_to_data(xuat_ra)
condition_to_data(thuyen_chuyen)
SCM_FG = pd.concat(nhap_vao + xuat_ra + thuyen_chuyen)

In [ ]:
print('RAW', SCM_RAW.shape)
print('FG', SCM_FG.shape)
print('WIP', SCM_WIP.shape)

In [ ]:
with pd.ExcelWriter('output/before_report.xlsx') as writer:  
    SCM_RAW.to_excel(writer, sheet_name='RAW')
    SCM_FG.to_excel(writer, sheet_name='FG')
    SCM_WIP.to_excel(writer, sheet_name='WIP')

In [ ]:
SCM_REPORT = SCM_RAW.pivot(index=["Unnamed: 0" ,"Material"], columns="Account code", values="Quantity")
SCM_REPORT.reset_index(inplace=True)
del SCM_REPORT['Unnamed: 0']
aggregation_functions = {
    101: sum, 102: 'sum', 
    201: sum, 261: 'sum', 
    343: sum, 401: 'sum',
    601: sum, 609: 'sum',
    720: sum, 721: 'sum',
}
SCM_REPORT = SCM_REPORT.groupby(SCM_REPORT['Material']).aggregate(aggregation_functions)

In [ ]:
col_list = SCM_REPORT.columns.values
col_list

In [ ]:
def get_result(df):
    result = df.pivot(index=["Unnamed: 0" ,"Material"], columns="Account code", values="Quantity")
    result.reset_index(inplace=True)
    del result['Unnamed: 0']
    aggregation_functions = {}
    columns_order = [101, 102, 321, 343, 401, 720, 201, 261, 344, 555, 601, 609, 721]
    for column in result.columns:
        if isinstance(column, int): 
            aggregation_functions[column] = 'sum'
    
    final_col = []
    for col in columns_order:
        if col in result.columns:
            final_col.append(col)
    
    result = result.groupby(result['Material']).aggregate(aggregation_functions)
#     columns_order = ['Material'] + final_col
    result = result[final_col]
    result.reset_index(inplace=True)
    result.index.name = None
    return result

In [ ]:
RAW_REPORT = get_result(SCM_RAW)
WIP_REPORT = get_result(SCM_WIP)
FG_REPORT = get_result(SCM_FG)

In [ ]:
with pd.ExcelWriter('output/final_result.xlsx') as writer:  
    RAW_REPORT.to_excel(writer, sheet_name='REPORT_RAW')
    WIP_REPORT.to_excel(writer, sheet_name='REPORT_FG')
    FG_REPORT.to_excel(writer, sheet_name='REPORT_WIP')

In [ ]:
RAW_REPORT

In [ ]:
score = [None]*65
for num in range(1, 65):
    score[num] = pd.read_excel(f'../data/Score 2021/{num:02}.xlsx')

In [ ]:
score.pop(0)
result = pd.concat(score)

In [ ]:
result

In [ ]:
result.to_excel('total_result.xlsx')

In [ ]:
my_filter = ((result['Văn'].notnull()) & (result['Lịch Sử'].notnull()) & (result['Địa Lý'].notnull()))

In [ ]:
khoi_c = result.loc[my_filter]

In [ ]:
khoi_a['A1'] = khoi_a['Toán'] + khoi_a['Lý'] + khoi_a['Ngoại Ngữ']
khoi_a['A'] = khoi_a['Toán'] + khoi_a['Lý'] + khoi_a['Hoá']

In [ ]:
khoi_a['TONG'] = khoi_a[["A1", "A"]].max(axis=1)

In [ ]:
# khoi_a.reset_index(inplace=True)

In [ ]:
khoi_a.sort_values(by='TONG', ascending=False, inplace=True)

In [ ]:
khoi_a.reset_index()

In [ ]:
khoi_a.to_excel('khoia.xlsx')

In [ ]:
khoi_a

In [ ]:
khoi_c['C'] = khoi_c['Văn'] + khoi_c['Lịch Sử'] + khoi_c['Địa Lý']

In [ ]:
khoi_c

In [ ]:
khoi_c.sort_values(by='C', ascending=False, inplace=True)

In [ ]:
khoi_c

In [ ]:
khoi_c.to_excel('khoi_c.xlsx')

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 45)
pd.set_option('display.max_rows', 50)

iob = pd.read_excel('../data/sample_data/IOB_sample_SW.xls', sheet_name='Ecus')

In [12]:
iob[['Ngày hóa đơn']].loc[pd.isnull(iob['Ngày hóa đơn'])]

,Ngày hóa đơn


In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 45)
pd.set_option('display.max_rows', 50)

In [14]:
hs = pd.read_excel('../data/sample_data/HS.xlsx', sheet_name='VIETNAM', dtype={'HS': object})

In [15]:
# hs = hs.iloc[3:]

In [16]:
# section = hs[hs.isnull().any(axis=1)]

In [19]:
# hs[hs['HS']==""] = np.NaN
hs.fillna(method='ffill', inplace=True)

In [20]:
hs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14362 entries, 0 to 14361
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   년도                       14362 non-null  int64 
 1   COUNTRY                  14362 non-null  object
 2   HS                       14362 non-null  object
 3   PRODUCT NAME VIETNAMESE  14362 non-null  object
 4   PRODUCT NAME ENGLISH     14362 non-null  object
 5   한글품명                     14362 non-null  object
 6   아세안-대한민국                 14360 non-null  object
 7   특혜세율                     14360 non-null  object
 8   아세안                      14360 non-null  object
 9   아세안-중국                   14360 non-null  object
 10  아세안-일본                   14360 non-null  object
 11  베트남-일본                   14360 non-null  object
 12  아세안-호주,
뉴질랜드             14360 non-null  object
 13  아세안-인도                   14360 non-null  object
 14  수입관세할당                   13685 non-nul

In [22]:
hs = hs[['HS', 'PRODUCT NAME VIETNAMESE']]

In [25]:
for _, row in hs.iterrows():
    print(row['HS'], row['PRODUCT NAME VIETNAMESE'])

0101 Ngựa, lừa, la sống.
0101 - Ngựa:
01012100 - - Loại thuần chủng để nhân giống
01012900 - - Loại khác
010130 - Lừa:
01013010 - - Loại thuần chủng để nhân giống
01013090 - - Loại khác
01019000 - Loại khác
0102 Động vật sống họ trâu bò.
0102 - Gia súc:
01022100 - - Loại thuần chủng để nhân giống
010229 - - Loại khác:
01022910 - - - Gia súc đực (kể cả bò đực)
01022990 - - - Loại khác
01022990 - Trâu:
01023100 - - Loại thuần chủng để nhân giống
01023900 - - Loại khác
010290 - Loại khác:
01029010 - - Loại thuần chủng để nhân giống
01029090 - - Loại khác
0103 Lợn sống.
01031000 - Loại thuần chủng để nhân giống
01031000 - Loại khác:
01039100 - - Trọng lượng dưới 50 kg
01039200 - - Trọng lượng từ 50 kg trở lên
0104 Cừu, dê sống.
010410 - Cừu:
01041010 - - Loại thuần chủng để nhân giống
01041090 - - Loại khác
010420 - Dê:
01042010 - - Loại thuần chủng để nhân giống
01042090 - - Loại khác
0105 Gia cầm sống, gồm các loại gà thuộc loài Gallus domesticus, vịt, ngan, ngỗng, gà tây và gà lôi.
0105

1702 Đường khác, kể cả đường lactoza, mantoza, glucoza và fructoza, tinh khiết về mặt hoá học, ở thể rắn; xirô đường chưa pha thêm hương liệu hoặc chất màu; mật ong nhân tạo đã hoặc chưa pha trộn với mật ong tự nhiên; đường caramen.
1702 - Lactoza và xirô lactoza:
17021100 - - Có hàm lượng lactoza khan từ 99% trở lên, tính theo trọng lượng chất khô
17021900 - - Loại khác
17022000 - Đường từ cây thích (maple) và xirô từ cây thích
170230 - Glucoza và xirô glucoza, không chứa hoặc có chứa hàm lượng fructoza không quá 20% tính theo trọng lượng ở thể khô:
17023010 - - Glucoza
17023020 - - Xirô glucoza
17024000 - Glucoza và xirô glucoza, chứa hàm lượng fructoza ít nhất là 20% nhưng dưới 50% tính theo trọng lượng ở thể khô, trừ đường nghịch chuyển
17025000 - Fructoza tinh khiết về mặt hoá học
170260 - Fructoza và xirô fructoza khác, chứa hàm lượng fructoza trên 50% tính theo trọng lượng ở thể khô, trừ đường nghịch chuyển:
17026010 - - Fructoza
17026020 - - Xirô fructoza
170290 - Loại khác, kể

30049020 - - Nước vô trùng đóng kín để xông, loại dược phẩm
30049030 - - Thuốc khử trùng
30049030 - - Chất gây tê, gây mê (Anaesthetic):
30049041 - - - Có chứa Procain hydrochloride
30049049 - - - Loại khác
30049049 - - Thuốc giảm đau, thuốc hạ sốt và các loại dược phẩm khác dùng để điều trị ho hoặc cảm lạnh, có hoặc không chứa chất kháng histamin:
30049051 - - - Chứa acetylsalicylic acid, paracetamol hoặc dipyrone (INN), dạng uống
30049052 - - - Chứa clorpheniramin maleat
30049053 - - - Chứa diclofenac, dạng uống
30049054 - - - Chứa piroxicam (INN) hoặc ibuprofen
30049055 - - - Loại khác, dạng dầu xoa bóp
30049059 - - - Loại khác
30049059 - - Thuốc chống sốt rét:
30049061 - - - Chứa artemisinin, artesunate hoặc chloroquine
30049062 - - - Chứa primaquine
30049062 - - - Loại khác:
30049063 - - - - Thuốc đông y từ thảo dược
30049069 - - - - Loại khác
30049069 - - Thuốc tẩy giun:
30049071 - - - Chứa piperazine hoặc mebendazole (INN)
30049071 - - - Loại khác:
30049072 - - - - Thuốc đông y 

44123200 - - Loại khác, với ít nhất một lớp mặt ngoài bằng gỗ không thuộc loại lá kim
44123900 - - Loại khác
44123900 - Loại khác:
44129400 - - Tấm khối, tấm mỏng và tấm lót
44129900 - - Loại khác
44130000 Gỗ đã được làm tăng độ rắn, ở dạng khối, tấm, thanh hoặc tạo hình.
44140000 Khung tranh, khung ảnh, khung gương bằng gỗ hoặc các sản phẩm bằng gỗ tương tự.
4415 Hòm, hộp, thùng thưa, thùng hình trống và các loại bao bì tương tự, bằng gỗ; tang cuốn cáp bằng gỗ; giá kệ để kê hàng, giá để hàng kiểu thùng và các loại giá để hàng khác, bằng gỗ; vành đệm giá kệ để hàng bằng gỗ.
44151000 - Hòm, hộp, thùng thưa, thùng hình trống và các loại bao bì tương tự; tang cuốn cáp
44152000 - Giá kệ để kê hàng, giá kệ để hàng kiểu thùng và các loại giá để hàng khác; vành đệm giá kệ để hàng
4416 Thùng tô nô, thùng tròn, thùng hình trống, hình trụ, có đai, các loại thùng có đai khác và các bộ phận của chúng, bằng gỗ, kể cả các loại tấm ván cong.
44160010 - Tấm ván cong
44160090 - Loại khác
4417 Dụng cụ c

6108 - Váy lót có dây đeo và váy lót trong:
61081100 - - Từ sợi nhân tạo
610819 - - Từ các vật liệu dệt khác:
61081920 - - - Từ lông cừu hoặc lông động vật mịn
61081930 - - - Từ bông
61081990 - - - Loại khác
61081990 - Quần xi líp và quần đùi bó:
61082100 - - Từ bông
61082200 - - Từ sợi nhân tạo
61082900 - - Từ các vật liệu dệt khác
61082900 - Váy ngủ và bộ pyjama:
61083100 - - Từ bông
61083200 - - Từ sợi nhân tạo
61083900 - - Từ các vật liệu dệt khác
61083900 - Loại khác:
61089100 - - Từ bông
61089200 - - Từ sợi nhân tạo
61089900 - - Từ các vật liệu dệt khác
6109 Áo phông, áo may ô và các loại áo lót khác, dệt kim hoặc móc.
610910 - Từ bông:
61091010 - - Dùng cho nam giới hoặc trẻ em trai
61091020 - - Dùng cho phụ nữ hoặc trẻ em gái
610990 - Từ các vật liệu dệt khác:
61099010 - - Dùng cho nam giới hoặc trẻ em trai, từ gai (ramie), lanh hoặc tơ tằm
61099020 - - Dùng cho nam giới hoặc trẻ em trai, từ các vật liệu khác
61099030 - - Dùng cho phụ nữ hoặc trẻ em gái
6110 Áo bó, áo chui đầu,

73079290 - - - Loại khác
730793 - - Loại hàn giáp mối:
73079310 - - - Có đường kính trong dưới 15 cm
73079390 - - - Loại khác
730799 - - Loại khác:
73079910 - - - Có đường kính trong dưới 15 cm
73079990 - - - Loại khác
7308 Các kết cấu bằng sắt hoặc thép (trừ nhà lắp ghép thuộc nhóm 94.06) và các bộ phận rời của các kết cấu (ví dụ, cầu và nhịp cầu, cửa cống, tháp, cột lưới, mái nhà, khung mái, cửa ra vào, cửa sổ, và các loại khung cửa, ngưỡng cửa ra vào, cửa chớp, lan can, cột trụ và các loại cột khác), bằng sắt hoặc thép; tấm, thanh, góc, khuôn, hình ống và các loại tương tự, đã được gia công để dùng làm kết cấu xây dựng, bằng sắt hoặc thép.
730810 - Cầu và nhịp cầu:
73081010 - - Dạng cấu kiện tiền chế được lắp ráp bằng các khớp nối
73081090 - - Loại khác
730820 - Tháp và cột lưới (kết cấu giàn):
730820 - - Tháp:
73082011 - - - Dạng cấu kiện tiền chế được lắp ráp bằng các khớp nối
73082019 - - - Loại khác
73082019 - - Cột lưới (kết cấu giàn):
73082021 - - - Dạng cấu kiện tiền chế được

84388012 - - Loại khác:
84388091 - - - Hoạt động bằng điện
84388092 - - - Không hoạt động bằng điện
843890 - Bộ phận:
843890 - - Của máy hoạt động bằng điện:
84389011 - - - Của hàng hóa thuộc phân nhóm 8438.30.10
84389012 - - - Của máy xay vỏ cà phê
84389019 - - - Loại khác
84389019 - - Của máy không hoạt động bằng điện:
84389021 - - - Của hàng hóa thuộc phân nhóm 8438.30.20
84389022 - - - Của máy sát tách vỏ cà phê
84389029 - - - Loại khác
8439 Máy chế biến bột giấy từ vật liệu sợi xenlulô hoặc máy dùng cho quá trình sản xuất hoặc hoàn thiện giấy hoặc bìa.
84391000 - Máy chế biến bột giấy từ vật liệu sợi xenlulô
84392000 - Máy dùng sản xuất giấy hoặc bìa
84393000 - Máy hoàn thiện sản phẩm giấy hoặc bìa
84393000 - Bộ phận:
84399100 - - Của máy sản xuất bột giấy từ vật liệu sợi xenlulô
84399900 - - Loại khác
8440 Máy đóng sách, kể cả máy khâu sách.
844010 - Máy:
84401010 - - Hoạt động bằng điện
84401020 - - Không hoạt động bằng điện
844090 - Bộ phận:
84409010 - - Của máy hoạt động bằng 

85329000 - Bộ phận
8533 Điện trở (kể cả biến trở và chiết áp), trừ điện trở nung nóng.
853310 - Điện trở than cố định, dạng kết hợp hay dạng màng:
85331010 - - Điện trở dán
85331090 - - Loại khác
85331090 - Điện trở cố định khác:
85332100 - - Có giới hạn công suất cho phép không quá 20 W
85332900 - - Loại khác
85332900 - Điện trở biến đổi kiểu dây quấn, kể cả biến trở và chiết áp:
85333100 - - Có giới hạn công suất cho phép không quá 20 W
85333900 - - Loại khác
85334000 - Điện trở biến đổi khác, kể cả biến trở và chiết áp
85339000 - Bộ phận
8534 Mạch in.
85340010 - Một mặt
85340020 - Hai mặt
85340030 - Nhiều lớp
85340090 - Loại khác
8535 Thiết bị điện để đóng ngắt mạch hay bảo vệ mạch điện, hoặc dùng để đấu nối hay lắp trong mạch điện (ví dụ, cầu dao, công tắc, cầu chì, bộ chống sét, bộ khống chế điện áp, bộ triệt xung điện, phích cắm, hộp đấu nối) dùng cho điện áp trên 1.000 V.
85351000 - Cầu chì
85351000 - Bộ ngắt mạch tự động:
853521 - - Có điện áp dưới 72,5 kV:
85352110 - - - Loại 

91112000 - Vỏ đồng hồ bằng kim loại cơ bản, đã hoặc chưa được mạ vàng hoặc mạ bạc
91118000 - Vỏ đồng hồ loại khác
91119000 - Bộ phận
9112 Vỏ đồng hồ thời gian và các loại tương tự dùng cho các mặt hàng khác của Chương này, và các bộ phận của chúng.
91122000 - Vỏ
91129000 - Bộ phận
9113 Dây đeo, quai đeo và vòng đeo đồng hồ cá nhân, và các bộ phận của chúng.
91131000 - Bằng kim loại quý hoặc kim loại dát phủ kim loại quý
91132000 - Bằng kim loại cơ bản, đã hoặc chưa mạ vàng hoặc bạc
91139000 - Loại khác
9114 Các bộ phận khác của đồng hồ thời gian hoặc đồng hồ cá nhân.
91141000 - Lò xo, kể cả dây tóc
91143000 - Mặt số
91144000 - Mâm và trục
91149000 - Loại khác
9201 Đàn piano, kể cả piano tự động; đàn clavecin (hapsichords) và các loại đàn dây có phím bấm khác.
92011000 - Đàn piano loại đứng
92012000 - Đại dương cầm
92019000 - Loại khác
9202 Các nhạc cụ có dây khác (ví dụ, ghi ta, vi-ô-lông, đàn hạc).
92021000 - Loại sử dụng cần kéo
92029000 - Loại khác
9205 Nhạc cụ hơi (ví dụ, các loại 